In [1]:
# from langchain.chat_models import ChatOpenAI
# from langchain.tools import StructuredTool
# from langchain.agents import initialize_agent, AgentType

# llm = ChatOpenAI(temperature=0.1)


# def plus(a, b):
#     return a + b


# # structured input ReAct(Reasoning and Acting) -> not related to reactjs lol
# # * how an agent works like a loop and langchain does pass the context.
# # langchane behaves like a bridge -> sometimes it can be broken
# # -> better option: OpenAI function calling -> force the output
# agent = initialize_agent(
#     llm=llm,
#     verbose=True,
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     tools=[
#         StructuredTool.from_function(
#             func=plus,
#             name="Sum Calculator",
#             description="A calculator that adds two numerical values and returns their sum. Both arguments must be numbers.",
#         )
#     ],
# )

# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# # llm.invoke(prompt)
# agent.run(prompt)
# # agent.invoke({"input": prompt})

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    # Deprecated since version 0.1.0: LangChain agents will continue to be supported,
    # but it is recommended for new use cases to be built with LangGraph.
    # https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    # It works well now without setting True
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="A calculator that adds two numerical values and returns their sum. Use this tool by sending a pair of numbers separated by a comma. \n Example:1,2",
        )
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"


# llm.invoke(prompt)
agent.run(prompt)
# agent.invoke({"input": prompt})



> Entering new AgentExecutor chain...
This is a large sum of money, so it would be best to use the Sum Calculator tool multiple times to add all the values together.
Action: Sum Calculator
Action Input: 355.39, 924.87
Observation: 355.39 924.87
Thought:I need to continue adding the previous sum to the next number in the list.
Action: Sum Calculator
Action Input: 1280.26, 721.2
Observation: 1280.26 721.2
Thought:I need to continue this process until I have added all the numbers together.
Action: Sum Calculator
Action Input: 2001.46, 1940.29
Observation: 2001.46 1940.29
Thought:I need to keep going until I have added all the numbers together.
Action: Sum Calculator
Action Input: 3941.75, 573.63
Observation: 3941.75 573.63
Thought:I need to keep adding the numbers together.
Action: Sum Calculator
Action Input: 4515.38, 65.72
Observation: 4515.38 65.72
Thought:I need to keep adding the numbers together.
Action: Sum Calculator
Action Input: 4581.1, 35.00
Observation: 4581.1 35.00
Thought:

'$5273.38'